In [ ]:
import pandas as pd

df = pd.read_csv('food_list.csv',encoding='cp949')

data_to_insert = {'구분': '_', '음식명': '_'}
df = df.append(data_to_insert, ignore_index=True)

data = df[['구분', '음식명']] # 필요한 데이터만 가져오기

for i in range (len(data)): # ,가 포함되거나 한 글자인 카테고리명 변경 / 문자가 있거나 한 글자인 음식명 변경
	if (',' in data['구분'][i]):
		data.loc[i, "구분"] = data['구분'][i].replace(',', '_')

	if (len(data['구분'][i]) == 1):
		data.loc[i, "구분"] = data['구분'][i] + "_"

	if (len(data['음식명'][i]) == 1):
		data.loc[i, "음식명"] = data['음식명'][i] + "_"

	if (" / " in data['음식명'][i]):
		data.loc[i, "음식명"] = data['음식명'][i].replace(" / ", '_')
	
	if ("/" in data['음식명'][i]):
		data.loc[i, "음식명"] = data['음식명'][i].replace("/", '_')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(1,2)) 
cv_category = cv.fit_transform(data['구분']) 
cv.vocabulary_ # 카테고리별 인덱스 번호 

cv.vocabulary_

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_category = cosine_similarity(cv_category, cv_category)
sim_category = similarity_category.argsort()[:,::-1]
print(sim_category)
sim_category.shape

In [ ]:
def recommend_sim_menu(data):

	cv = CountVectorizer(ngram_range=(1,2)) 
	cv_menuname = cv.fit_transform(data['음식명'])
	similarity_menuname = cosine_similarity(cv_menuname, cv_menuname)
	sim_menuname = similarity_menuname.argsort()[:,::-1]
	return (similarity_menuname)
	sim_menuname.shape

In [ ]:
def recommend_sim(similarity_menuname):

	cate_menu_co = (
		#+ similarity_category * 0.8
		+ similarity_menuname #* 0.2
	)

	cate_menu_co_idx = cate_menu_co.argsort()[:,::-1]
	return (cate_menu_co_idx)

In [ ]:
def remove_in_list(menu_list):
	for i in range (len(menu_list)):
		menu_list[i] = menu_list[i].replace('_', '')
	return menu_list

In [ ]:
import numpy as np

def recommend_menu(df, sorted_idx, menu_name, top=15):
	lst = []
	
	# 유사도 계산해서 유사도순으로 추천

	for i in range (len(data)):
		lst.append(menu_name in df['음식명'][i])

	result = df.iloc[lst]
	return result['음식명'].values ## 추천 메뉴 전송 필요


	#target_menu_idx = df[df['음식명'] == menu_name].index.values

	#sim_idx = sorted_idx[target_menu_idx, :top].reshape(-1)

	#result = df.iloc[sim_idx]
	#return result


In [ ]:
menu_name = "김치볶" ## 입력

lst = []

if (len(menu_name) == 1):
	menu_name += "_"

if (True in (data['음식명'] == menu_name).values): # 입력과 일치하는 메뉴가 있는 경우
		print("잂치하는 메뉴가 있습니다.")
		lst.append(menu_name)
		
else : # 입력과 일치하는 메뉴명이 없는 경우
	lst = recommend_menu(data, cate_menu_co_idx, menu_name)
	if (len(lst) != 0):
		print("일치하는 메뉴가 없습니다. 추천 메뉴는 다음과 같습니다.")

if (len(lst) == 0):
	print("메뉴가 없습니다.")
else :
	lst = remove_in_list(lst)
	print(lst) ## 주문 메뉴 전송 필요

In [ ]:
import numpy as np

def recommend_menu_(df, menu_name, top=15):
	last = len(df) - 1
	df.loc[last, "음식명"] = menu_name

	similarity_menuname = recommend_sim_menu(df)
	sorted_idx = recommend_sim(similarity_menuname)
	
	sim_idx = sorted_idx[[last], :top].reshape(-1)

	result = df.iloc[sim_idx]
	return result

In [ ]:
menu_name = "햄버거"

if (len(menu_name) == 1):
	menu_name += "_"

print(recommend_menu_(data, menu_name))